# Fake job Prediction: Investigating the pattern to avoid the job scam

# Notebook 3 -Text Pre-processing

**Third setp: Text field pre process for NLP.**



**By: Polly Pang**


In this portion of the notebook, I will continuing working on cleaning methods especially text columns. I will be modifying columns, combine columns, and preparing the data for the implementation of Natural Language Processing (NLP) and other Machine Learning techniques.

---

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home"> Contents</h3>

* [1. Libraries and load data](#1)
* [2. Text columns](#2)
    - [2.1 clean text columns](#2.1)
    - [2.2 combine text columns](#2.2)
    - [2.3 drop redundant columns](#2.3)
* [3. Save File](#3)
* [4. End of Notebook 3](#4)

## 1. Libraries <a id="1"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import re


#from langdetect import detect

In [25]:
# load data
text_clean_df = joblib.load('data/fake_job_cleaned.pkl')

In [26]:
text_clean_df.head()

,title,department,has_salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,country,province,city
0,Marketing Intern,Marketing,0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Blank,0,1,0,Other,Internship,Blank,Blank,Marketing,0,US,NY,New York
2,Commissioning Machinery Assistant (CMA),Blank,0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,Blank,0,1,0,Blank,Blank,Blank,Blank,Blank,0,US,IA,Wever
3,Account Executive - Washington DC,Sales,0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington
4,Bill Review Manager,Blank,0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth
5,Accounting Clerk,Blank,0,Blank,Job OverviewApex is an environmental consultin...,Blank,Blank,0,0,0,Blank,Blank,Blank,Blank,Blank,0,US,MD,Blank


# 2. Text columns<a id="2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

**Goal**:
- My business goal is to detect the tect data from the job posts, and finding out is this is a fraudulent. I kept all columns initially to showcase some cleaning steps and EDA that may be used in later analysis. However, I will be dropping some columns to focus on my corpus, they will be the metadata for extend study or model improvement.
column: `text` will includes all the text column which I want to use in the model.
----

## 2.1 clean text columns<a id="2.1"></a>

**plan**:
- `text`:'country', 'title', 'department', 'company_profile', 'description', 'requirements','benefits',  'employment_type','industry', 'function','required_experience','required_education.
- columns `city` and `provience` will not included as I want to keep the sparse matrixes samll, I will only keep the top layer of the location. I will drop column `city` and `provience`
- **clean**:
    - white space
    - separate sentences 
    - remove URLs
    - non-ascii

In [53]:
text_columns=['country', 'title', 'department', 'company_profile', 'description', 'requirements','benefits', 'employment_type','industry', 'function','required_experience','required_education']

In [54]:
for col in text_columns:
    text_clean_df[col] = text_clean_df[col].apply(lambda x: re.sub(r'â€™', '', str(x))) # remove specific character used for apostrophe
                                                              # replace with '' before ascii removal so contractions together
                                                            # works for a.A as well 
    text_clean_df[col] = text_clean_df[col].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in str(x)])) # remove non-ascii
    text_clean_df[col] = text_clean_df[col].apply(lambda x: re.sub('http[^\s]+ ', ' ', str(x))) # remove URLs
    text_clean_df[col] = text_clean_df[col].apply(lambda x: re.sub('url[^\s]+ ', ' ', str(x)))# remove URLs
    text_clean_df[col] = text_clean_df[col].apply(lambda x: re.sub('URL[^\s]+ ', ' ', str(x)))# remove URLs
    text_clean_df[col] = text_clean_df[col].apply(lambda x: re.sub(' +', ' ', x)) # remove double and triple spaces
    text_clean_df[col] = text_clean_df[col].apply(lambda x: str(x).strip()) # remove white space trailing/leading

In [55]:
text_clean_df.sample(5) # Check cleaning results

,title,department,has_salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,country,province,city
17634,URGENT Part Timers & Full Timers Required.,Blank,0,Blank,URGENT Part Timers &amp; Full Timers Required....,Blank,Blank,0,0,0,Part-time,Blank,Blank,Blank,Blank,1,AU,NSW,Sydney
10801,NYC Field Organizer,Marketing,0,Farmigo is a mission-driven startup with a sim...,Farmigo is hiring Field Organizers to join our...,"ResponsibilitiesAs a Field Organizer, you will...","Full-time, salaried position. Full health bene...",0,1,1,Full-time,Blank,Bachelor's Degree,Blank,Marketing,0,US,NY,Brooklyn
3813,The Untraditional Community Manager,Blank,0,We are the creators of Scypho - a revolutionar...,Join the Evolution and take on the role as com...,Proficient in HTML/CSS/JavaScriptB.S. universi...,Be part of an awesome team and one of the hott...,0,1,1,Full-time,Not Applicable,Bachelor's Degree,Consumer Electronics,Marketing,0,SE,Blank,Stockholm
3594,Client Services Co-ordinator,Client Services,1,Indigo Clothing produces customised clothing f...,The Client Services Co-ordinator will support ...,Excellent Telephone MannerAbility to deal with...,Base salary dependent on experience (between 1...,0,1,1,Contract,Entry level,Bachelor's Degree,Printing,Project Management,0,GB,LND,Leyton
6477,Senior Java Engineer,Blank,0,We re always looking for highly motivated foun...,The Mobile Majority is a rapidly growing ad te...,We're looking for an engineer who has knowledg...,"It s no secret that we work hard, but we also ...",0,1,1,Full-time,Mid-Senior level,Blank,Internet,Engineering,0,US,CA,Santa Monica


## 2.2 combine text columns<a id="2.2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

**plan**:
- combine columns: `country`, `title`, `department`, `company_profile`, `description`, `requirements`,`benefits`, `employment_type`,`industry`, `function`.

In [60]:
# combine the text columns
text_clean_df['text']=text_clean_df['title']+' '+text_clean_df['department']+' '+text_clean_df['company_profile']+' '+text_clean_df['description']+' '+\
text_clean_df['requirements']+' '+text_clean_df['benefits']+' '+text_clean_df['employment_type']+' '+text_clean_df['required_experience']+' '+text_clean_df['required_education']+\
text_clean_df['industry']+' '+text_clean_df['function']+' '+text_clean_df['country']

In [61]:
# check the text data sample
text_clean_df.head()['text'][0]

"Marketing Intern Marketing We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial, business, and engineering team. We're focused on using technology to find new and better ways to connect people around their specific food interests, and to offer them superb, highly curated information about food and cooking. We attract the most talented home cooks and contributors in the country; we also publish well-known professionals like Mario Batali, Gwyneth Paltrow, and Danny Meyer. And we have partnerships with Whole Foods Market and Random House.Food52 has been named the best food website by the James Beard Foundation and IACP, and has been featured in the New York Times, NPR, Pando Daily, TechCrunch, and on the Today Show.We're located in Chelsea, in New York City. Food52, a fast-growing, James Beard Award-winning online food community and crowd-sour

## 2.3 drop redundant columns<a id="2.3"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [62]:
print(f"columns need to be dropped: {text_columns}")

columns need to be dropped: ['country', 'title', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'industry', 'function', 'required_experience', 'required_education']


In [92]:
df_model_full=text_clean_df.copy()
df_model_full.drop(text_columns,inplace=True,axis=1)
df_model_full.drop(['city','province'],inplace=True,axis=1)
df_model_full.reset_index(drop=True,inplace=True)


In [93]:
df_model_full.head()

,has_salary_range,telecommuting,has_company_logo,has_questions,fraudulent,text
0,0,0,1,0,0,"Marketing Intern Marketing We're Food52, and w..."
1,0,0,1,0,0,Commissioning Machinery Assistant (CMA) Blank ...
2,0,0,1,0,0,Account Executive - Washington DC Sales Our pa...
3,0,0,1,1,0,Bill Review Manager Blank SpotSource Solutions...
4,0,0,0,0,0,Accounting Clerk Blank Blank Job OverviewApex ...


In [3]:
df_model_full.shape

(17879, 6)

- The shape of my final dataset is (17879, 6)

# 3.Save File<a id="3"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [94]:
# Save df_model_full df as pickle file
joblib.dump(df_model_full, 'data/df_model_full_df.pkl')

['data/df_model_full_df.pkl']

-----

**Approach 1:**
- Focus on `text` only, keep the other columns as the matadata.

In [97]:
text_df=df_model_full.copy()
text_df.drop(['has_salary_range','telecommuting','has_company_logo','has_questions'],axis=1,inplace=True)
text_df.reset_index(drop=True,inplace=True)

In [98]:
text_df.head()

,fraudulent,text
0,0,"Marketing Intern Marketing We're Food52, and w..."
1,0,Commissioning Machinery Assistant (CMA) Blank ...
2,0,Account Executive - Washington DC Sales Our pa...
3,0,Bill Review Manager Blank SpotSource Solutions...
4,0,Accounting Clerk Blank Blank Job OverviewApex ...


In [99]:
# Save df_model_full df as pickle file
joblib.dump(text_df, 'data/df_text_model_df.pkl')

---

# 4. End of Notebook 3 <a id="4"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

This is the end of Notebook 3.
In the next notebook, I will be performing some Modeling. See Notebook 4-Modeling.